# Imports

In [1]:
# Standard library imports
import sys
import os

# Add path
sys.path.insert(0, os.path.join('D:\\','OneDrive','Dokumente','Python','BMCSim'))

# Third party imports
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

# Local application imports
from BMCtool import BMCTool

%matplotlib qt

# settings

In [2]:
# simulation settings
n_timesteps = 200  # number of timesteps for each RF pulse
track = True  # track the magnetization trajectory during pulses/delays ?!
par_calc = True  # calculate all offsets in parallel instead of subsequently

# experimental settings
B0 = 3  # B0 in T
B1 = 1e-6  # B1 in µT (for WASABI use 1.25e-6*B0)
shape = 'siemens_gauss'  # shape of saturation pulse
n_p = 5  # number of saturation pulses
tp = 0.2  # pulse duration in s (for WASABI use 0.015/B0)
td = 0.1  # delay between pulses in s
trec = 0  # recovery/delay time between offsets in s
offset = 1  # max offset in ppm
n_offsets = 1  # number of offsets

# sample settings
T1 = 2            #T1 in s
T2 = 0.2            #T2 in s

# CEST settings
n_pools = 2

#                bulk       1st       2nd       3rd       4th       5th       6th
T1n = np.array([   T1,       T1,       T1,       T1,       T1,       T1,       T1])
T2n = np.array([   T2,     0.15,     9e-6,     0.15,    0.015,     9e-6,    0.015])
fn  = np.array([    1,    0.002,     0.14,    0.005,    0.005,     0.14,     0.01])
dwn = np.array([ -0.0,        1,     -2.6,     -3.3,     -3.7,     -2.6,     -1.0]) * 42.577 * 2 * np.pi * B0
kn  = np.array([    0,      100,       40,     2000,     2000,       40,      500])

# calculate offsets
offsets_ppm = np.linspace(-offset, offset, n_offsets)
#offsets_ppm = np.linspace(110, 130, n_offsets)
offsets = offsets_ppm * 42.577 * 2 * np.pi * B0
if len(offsets)==1:
    offsets=-offsets

# simulation

In [3]:
# create Sim object
Sim = BMCTool(b0=B0,
              n_pools=n_pools,
              t1_values=T1n,
              t2_values=T2n,
              poolsizes=fn,
              resonances=dwn,
              exchange_rates=kn,
              track=track)

N = 1 if par_calc else len(offsets)
m = len(offsets) if par_calc else 1

M_ = np.zeros([len(offsets), 3*n_pools, 1])

# set initial magnetization
Sim.set_M(m)

for i in range(N):
    x = offsets if par_calc else offsets[i]
    
    # simulate recovery
    if trec != 0:
        Sim.solve(x, 0, trec, int(trec/tp*n_timesteps), shape='Pause')

    # simulate saturation
    for n in range(n_p):
        if n != 0 and td != 0:
            Sim.solve(x, b1=0, pulse_dur=td, steps=int(td/tp*n_timesteps), shape='Pause')
        Sim.solve(x, b1=B1, pulse_dur=tp, steps=n_timesteps, shape=shape)

    if not par_calc:
        M_[i,] = Sim.M_

        
# get magnetization vector
if par_calc:
    M = Sim.M_
else:
    M = M_

In [4]:
fig, ax = plt.subplots(figsize=(12,9))
ax.plot(Sim.t, Sim.amp_)

# Plot time curve (Signal vs. time)

In [5]:
fig, ax = plt.subplots()

idx = int(np.where(offsets == offsets[np.abs(offsets - dwn[1]).argmin()])[0])

ax.plot(Sim.t, Sim.Mt[:,idx,2,0], marker=None, markersize=8, linestyle='-', linewidth=2)
ax.set_xlabel('time [s]', fontsize=20)
ax.set_ylabel('normalized signal', fontsize=20)

Text(0, 0.5, 'normalized signal')

# Plot magnetization trajectory

In [6]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.set_xlim3d(-1.1, 1.1)
ax.set_ylim3d(-1.1, 1.1)
ax.set_zlim3d(   0, 1.1)

ax.plot3D(Sim.Mt[:,idx,0,0], Sim.Mt[:,idx,1,0], Sim.Mt[:,idx,2,0], 'gray')
#ax.scatter(Mt[:,0,0,0], Mt[:,0,1,0], Mt[:,0,2,0], zdir='z')


In [7]:
xxx = np.zeros(len(Sim.Mt[1:,0,0,0]))
print(len(xxx))
ax.quiver(xxx,xxx,xxx,
          Sim.Mt[1:,0,0,0], Sim.Mt[1:,0,1,0], Sim.Mt[1:,0,2,0],
          arrow_length_ratio=0.1,
          pivot='tail')

1399


# Magnetization Video

In [8]:
matplotlib.use("Agg")

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(elev=30, azim=-60)

In [9]:
fig.gca().cla()

tt = Sim.t
xx = Sim.Mt[:,idx,0,0]
yy = Sim.Mt[:,idx,1,0]
zz = Sim.Mt[:,idx,2,0]

ax.set_xlim3d(-1.1, 1.1)
ax.set_ylim3d(-1.1, 1.1)
ax.set_zlim3d(   0, 1.1)

ax.set_axis_off()

x_ax = ax.quiver(-1.1,0,0,2.2,0,0, color='grey', arrow_length_ratio=0.05)
y_ax = ax.quiver(0,1.1,0,0,-2.2,0, color='grey', arrow_length_ratio=0.05)
z_ax = ax.quiver(0,0,0,0,0,1.2, color='grey', arrow_length_ratio=0.1)

b_vec = ax.quiver(0,0,0,0,0,0,
                  length=1.1, normalize=True, 
                  color='black',
                  lw=2,
                  arrow_length_ratio=0.1)
   
line, = ax.plot([0,0], [0,0], [0,0], color='red')
          
vec = ax.quiver(0,0,0,0,0,0,
                arrow_length_ratio=0.1,
                pivot='tail',
                color='yellow')

def update(t):
    
    line.set_data(xx[:t], yy[:t])
    line.set_3d_properties(zz[:t])
    
    global vec
    vec.remove()   
    vec = ax.quiver(0,0,0,xx[t],yy[t],zz[t],
                    arrow_length_ratio=0.1,
                    pivot='tail',
                    color='blue')
    
    global b_vec
    b_vec.remove()
    #print(42.577e6*2*np.pi*Sim.amp_[t])
    #print(offsets[idx])
    b_vec = ax.quiver(0,0,0,42.577e6*2*np.pi*Sim.amp_[t], 0, offsets[idx],
                  length=1.1, normalize=True, 
                  color='black',
                  lw=2,
                  arrow_length_ratio=0.1)

    return (line,)
    
# Set up formatting for the movie files
#Writer = animation.writers['ffmpeg']
#writer = Writer(fps=1000, bitrate=1800)

ani = animation.FuncAnimation(fig, update, len(xx), interval=20)

In [10]:
#ani.save('WASABI_-2ppm.mp4', writer=writer)
HTML(ani.to_html5_video()) # show as video player in notebook

RuntimeError: Requested MovieWriter (ffmpeg) not available

In [11]:
matplotlib.use("Agg")

fig2 = plt.figure(figsize=(12,9))
ax2 = fig2.add_subplot(111)

In [12]:
fig2.gca().cla()

tt = Sim.t
zz = Sim.Mt[:,0,2,0]
print(len(tt))

ax2.set_xlim(0, np.max(tt))
ax2.set_ylim(0, 1.1)
line2, = ax2.plot([0,0], [0,0], color='red')
          
def update2(t):
    
    line2.set_data(tt[:t], zz[:t])
    return (line2,)
    
# Set up formatting for the movie files
Writer2 = animation.writers['ffmpeg']
writer2 = Writer2(fps=30, bitrate=1800)

ani2 = animation.FuncAnimation(fig2, update2, len(tt))

1400


RuntimeError: Requested MovieWriter (ffmpeg) not available

In [ ]:
#ani2.save('Z_vs_time.mp4', writer=writer)
HTML(ani2.to_html5_video()) # show as video player in notebook